## Required Packages

In [2]:
from enum import Enum
from math import ceil
from typing import Optional
import os, time, logging, pandas as pd, re
from urllib.parse import quote_plus

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import (
    TimeoutException, NoSuchElementException,
    NoSuchWindowException, WebDriverException,
    ElementClickInterceptedException, StaleElementReferenceException
)

from IPython.display import display

logging.basicConfig(level=logging.INFO, format="%(levelname)s: %(message)s")



class ExperienceFilter(Enum):
    ALL = ""
    INTERNSHIP = "1"
    ENTRY_LEVEL = "2"
    ASSOCIATE = "3"
    MID_SENIOR_LEVEL = "4"
    DIRECTOR = "5"
    EXECUTIVE = "6"

class JobPostTime(Enum):
    ANY_TIME = ""
    PAST_24_HOURS = "r86400"
    PAST_WEEK = "r604800"
    PAST_MONTH = "r2592000"


/Users/50357691/Desktop/Job_Skill_Gap_Analysis/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


## Use chrome driver to search for job postinigs

In [3]:
# Selenium is a automation tool that can control a web browser. It could do human actions in web pages like click, scroll, input text, etc. Which can be used to bypass some anti-scraping mechanisms.

driver = None

def make_driver(headless: bool = False):
    opts = Options()
    if headless:
        opts.add_argument("--headless=new")
    opts.add_argument("--lang=en-US")
    return webdriver.Chrome(options=opts)

def ensure_window(drv, headless: bool = False):
    global driver
    if drv is None:
        driver = make_driver(headless=headless)
        return driver
    try:
        _ = drv.current_url
        if not drv.window_handles:
            raise NoSuchWindowException("No window handles")
        return drv
    except Exception:
        logging.warning("Driver/window lost. Recreating browser...")
        try:
            drv.quit()
        except Exception:
            pass
        driver = make_driver(headless=headless)
        return driver

def safe_get(drv, url: str, retries: int = 1, headless: bool = False):
    global driver
    for _ in range(retries + 1):
        try:
            drv.get(url)
            return drv
        except WebDriverException as e:
            logging.warning(f"driver.get failed ({e.__class__.__name__}); recreating…")
            drv = ensure_window(drv, headless=headless)
            driver = drv
    return drv


In [4]:
from getpass import getpass

def try_login(drv, email: str, password: str) -> None:
    global driver
    driver = ensure_window(drv)
    driver = safe_get(driver, "https://www.linkedin.com/login")
    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, "username")))
    u = driver.find_element(By.ID, "username"); u.clear(); u.send_keys(email)
    p = driver.find_element(By.ID, "password"); p.clear(); p.send_keys(password); p.send_keys(Keys.RETURN)
    WebDriverWait(driver, 25).until(
        EC.any_of(
            EC.presence_of_element_located((By.ID, "global-nav-search")),
            EC.presence_of_element_located((By.ID, "input__email_verification_pin")),
            EC.url_contains("checkpoint")
        )
    )

def _maybe_accept_cookies_and_check_login(drv) -> bool:
    for sel in [
        "button[aria-label*='Accept']",
        "button[data-test-global-nav-cookie-banner-accept]",
        ".artdeco-global-alert-action button",
    ]:
        try:
            for b in drv.find_elements(By.CSS_SELECTOR, sel):
                if b.is_displayed() and b.is_enabled():
                    b.click()
        except Exception:
            pass
    cur = drv.current_url
    if "login" in cur or "checkpoint" in cur:
        return False
    try:
        WebDriverWait(drv, 6).until(EC.presence_of_element_located((By.CSS_SELECTOR, "main, body")))
        return True
    except Exception:
        return False

def login_wall_present(drv):
    try:
        modals = drv.find_elements(By.CSS_SELECTOR, '.artdeco-modal, [role="dialog"]')
        return any(m.is_displayed() for m in modals)
    except Exception:
        return False


# LinkedIn has many different languages, so the dismiss button could be in different languages. Here we try to cover some common ones.
def dismiss_login_wall(drv, timeout=3) -> bool:
    selectors = [
        'button.artdeco-modal__dismiss',
        'button[aria-label*="Dismiss"]',
        'button[aria-label*="Close"]',
        'button[aria-label*="关闭"]',
        'button[aria-label*="關閉"]',
    ]
    for sel in selectors:
        try:
            btn = WebDriverWait(drv, timeout).until(EC.element_to_be_clickable((By.CSS_SELECTOR, sel)))
            drv.execute_script("arguments[0].click();", btn)
            WebDriverWait(drv, 5).until_not(EC.presence_of_element_located((By.CSS_SELECTOR, '.artdeco-modal, [role="dialog"]')))
            return True
        except Exception:
            pass
    try:
        drv.find_element(By.TAG_NAME, "body").send_keys(Keys.ESCAPE)
        WebDriverWait(drv, 2).until_not(EC.presence_of_element_located((By.CSS_SELECTOR, '.artdeco-modal, [role="dialog"]')))
        return True
    except Exception:
        return False

# Credentials 
EMAIL = "zh272@georgetown.edu"

# Protect your password
PASSWORD = os.getenv("LI_PASSWORD") or getpass("Enter your LinkedIn password (hidden): ")


In [5]:
# CSS selectors for job listings container and items, which help to locate job postings on the page.
CONTAINER_SELECTORS = [
    "div.jobs-search-results-list",
    ".scaffold-layout__list > div",
    "div.jobs-search-two-pane__results",
    "div.jobs-search-results-list__container",
    "section.two-pane-serp-page__results-list",
]

ITEM_SELECTORS = [
    "li[data-occludable-job-id]",
    "ul.jobs-search-results__list > li",
    "li.jobs-search-results__list-item",
    "div.jobs-search-results__list-item",
    "ul.jobs-search__results-list li",
    "div.base-card, div.job-card-container",
]


def wait_for_job_items(drv, timeout=20):
    any_sel = ", ".join(ITEM_SELECTORS)
    WebDriverWait(drv, timeout).until(EC.presence_of_element_located((By.CSS_SELECTOR, any_sel)))

def safe_text(el):
    return el.text.strip() if el else ""

def find_one(drv, *locator, timeout=8, visibility=True):
    cond = EC.visibility_of_element_located if visibility else EC.presence_of_element_located
    try:
        return WebDriverWait(drv, timeout).until(cond(locator))
    except Exception:
        return None

def get_title_company_location(drv):
    # Title
    title = ""
    for sel in [".job-view-layout h1", "h1.jobs-unified-top-card__job-title", "h1"]:
        els = drv.find_elements(By.CSS_SELECTOR, sel)
        if els and els[0].text.strip():
            title = els[0].text.strip(); break

    # Company
    company = ""
    for sel in [
        "[data-view-name='job-details-about-company-name-link']",
        "a.jobs-unified-top-card__company-name",
        ".jobs-unified-top-card__company-name a",
        ".jobs-unified-top-card__company-name",
    ]:
        els = drv.find_elements(By.CSS_SELECTOR, sel)
        if els and els[0].text.strip():
            company = els[0].text.strip(); break

    # Location: keep Remote/Hybrid/On-site and specific city/region. If it only has country, then keep empty
    import re
    location = ""
    sel_list = [
        ".jobs-unified-top-card__primary-description-container li",
        ".jobs-unified-top-card__primary-description-container span",
        ".job-details-jobs-unified-top-card__primary-description-container li",
        ".job-details-jobs-unified-top-card__primary-description-container span",
        ".jobs-unified-top-card__subtitle-primary-grouping span",
        ".jobs-unified-top-card__subtitle-primary-grouping",
        ".jobs-unified-top-card__bullet",
        ".jobs-unified-top-card__workplace-type",
    ]
    candidates = []
    for sel in sel_list:
        for e in drv.find_elements(By.CSS_SELECTOR, sel):
            t = e.text.strip()
            if t:
                candidates.append(t)

    for t in candidates:
        low = t.lower()
        if any(k in low for k in ["remote", "hybrid", "on-site", "onsite"]):
            location = t; break

    if not location:
        for t in candidates:
            if re.search(r".+,\s+.+", t):  
                location = t; break

    if location and location.strip().lower() in {"united states", "usa"}:
        location = ""

    return title, company, location


In [6]:
# Most job descriptions are partially hidden and need to be expanded by clicking "See more" or similar buttons. This function tries to find and click those buttons to reveal the full job description.
def expand_all_in(drv, root=None, max_clicks=10):
    selectors = [
        "button.show-more-less-html__button",
        "button[aria-label*='See more']",
        "button[aria-label*='Show more']",
        "button[aria-controls*='description']",
        "button[aria-expanded='false']",
        "button[aria-label*='显示更多']",
        "button[aria-label*='顯示更多']",
    ]
    did = False
    for _ in range(max_clicks):
        clicked_this_round = False
        scope = root if root is not None else drv
        for sel in selectors:
            for b in scope.find_elements(By.CSS_SELECTOR, sel):
                try:
                    if not b.is_displayed() or not b.is_enabled():
                        continue
                    drv.execute_script("arguments[0].scrollIntoView({block:'center'});", b)
                    time.sleep(0.15)
                    drv.execute_script("arguments[0].click();", b)
                    did = True; clicked_this_round = True
                    time.sleep(0.2)
                except Exception:
                    pass
        if not clicked_this_round:
            break
    return did

def scroll_until_all_jobs_load(drv, container, pause=0.8, max_tries=30, item_selector="li.jobs-search-results__list-item"):
    tries = 0
    last_height = drv.execute_script("return arguments[0].scrollHeight", container)
    while tries < max_tries:
        drv.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", container)
        time.sleep(pause)
        new_height = drv.execute_script("return arguments[0].scrollHeight", container)
        if new_height == last_height:
            break
        last_height = new_height; tries += 1
    items = drv.find_elements(By.CSS_SELECTOR, item_selector)
    if not items:
        items = drv.find_elements(By.CSS_SELECTOR, "ul.jobs-search-results__list > li, li[data-occludable-job-id], ul.jobs-search__results-list li")
    return items


In [7]:
def _first_text(root, selectors):
    for sel in selectors:
        els = root.find_elements(By.CSS_SELECTOR, sel)
        if els:
            txt = els[0].text.strip()
            if txt:
                return txt
    return ""

def _first_el(root, selectors):
    for sel in selectors:
        els = root.find_elements(By.CSS_SELECTOR, sel)
        if els:
            return els[0]
    return None

def _first_attr(root, selectors, attr):
    el = _first_el(root, selectors)
    return el.get_attribute(attr) if el else ""


In [8]:
# Extract the jobs data from the job listings
def populate_jobs_data_minimal(jobs, nbOfJobs):
    job_data = {"Title": [], "Company": [], "Location": [], "Link": [], "Description": []}
    totalJobs = min(nbOfJobs, len(jobs))
    list_handle = driver.current_window_handle

    for idx, job in enumerate(jobs[:totalJobs]):
        try:
            link_el = _first_el(job, ["a.base-card__full-link", "a.job-card-container__link", "a[href*='/jobs/view/']"])
            if not link_el: continue
            link = link_el.get_attribute("href") or ""
            driver.execute_script("window.open(arguments[0], '_blank');", link)
            WebDriverWait(driver, 10).until(lambda d: len(d.window_handles) > 1)
            driver.switch_to.window(driver.window_handles[-1])

            WebDriverWait(driver, 12).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".jobs-unified-top-card, .job-view-layout, h1")))
            expand_all_in(driver, None, max_clicks=8)

            title, company, location = get_title_company_location(driver)

            desc_root = None
            for sel in ["#job-details", ".jobs-description__content", ".jobs-box__html-content", ".show-more-less-html__markup"]:
                found = driver.find_elements(By.CSS_SELECTOR, sel)
                if found: desc_root = found[0]; break
            if desc_root: expand_all_in(driver, desc_root, max_clicks=5)

            description = desc_root.text.strip() if desc_root else ""

            job_data["Company"].append(company)
            job_data["Title"].append(title)
            job_data["Location"].append(location)
            job_data["Link"].append(link)
            job_data["Description"].append(description)

        except Exception as e:
            logging.warning(f"[details-minimal] job {idx} skipped: {e}")
        finally:
            if len(driver.window_handles) > 1:
                driver.close()
                driver.switch_to.window(list_handle)
                if login_wall_present(driver):
                    dismiss_login_wall(driver)

    return pd.DataFrame(job_data)


# read data from the job cards in the listing page, without visiting each job's detail page
def extract_from_list_cards_min(max_items: int = 25) -> pd.DataFrame:
    cards = driver.find_elements(By.CSS_SELECTOR, "ul.jobs-search__results-list li")
    if not cards:
        cards = driver.find_elements(By.CSS_SELECTOR, "ul.jobs-search-results__list li")
    if not cards:
        cards = driver.find_elements(By.CSS_SELECTOR, "div.base-card, div.job-card-container")

    data = {"Title": [], "Company": [], "Location": [], "Link": [], "Description": []}
    for c in cards[:max_items]:
        try:
            title = _first_text(c, [".base-search-card__title", ".job-card-list__title", "h3"])
            company = _first_text(c, [
                ".base-search-card__subtitle a", ".base-search-card__subtitle",
                ".job-card-container__company-name", ".job-search-card__subtitle a",
                ".job-search-card__subtitle"
            ])
            location = _first_text(c, [
                ".job-search-card__location", "span.job-search-card__location",
                "li.job-card-container__metadata-item"
            ])
            link = _first_attr(c, ["a.base-card__full-link", "a.job-card-container__link", "a[href*='/jobs/view/']"], "href")

            if title or company or link:
                data["Title"].append(title); data["Company"].append(company); data["Location"].append(location)
                data["Link"].append(link); data["Description"].append("")
        except Exception:
            continue

    return pd.DataFrame(data)


In [9]:
# Scrape jobs from the current page
def scrape_jobs_from_page(nb_to_scrape: int) -> pd.DataFrame:
    if login_wall_present(driver):
        dismissed = dismiss_login_wall(driver)
        if not dismissed:
            print("Login wall present; cannot dismiss. Try logging in.")
            return pd.DataFrame()

    try:
        wait_for_job_items(driver, timeout=20)
    except TimeoutException:
        print("⚠️ No job cards visible (timeout).")
        return pd.DataFrame()

    container = None
    for sel in CONTAINER_SELECTORS:
        try:
            container = driver.find_element(By.CSS_SELECTOR, sel); break
        except Exception:
            continue

    for _ in range(2):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);"); time.sleep(0.4)
    if container:
        for _ in range(2):
            driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight;", container); time.sleep(0.4)

    items = []
    for sel in ITEM_SELECTORS:
        found = driver.find_elements(By.CSS_SELECTOR, sel)
        if found:
            items = found; break

    if container:
        try:
            items = scroll_until_all_jobs_load(driver, container)
        except Exception:
            pass

    if items:
        df = populate_jobs_data_minimal(items, nb_to_scrape)
        if not df.empty:
            return df

    df2 = extract_from_list_cards_min(max_items=nb_to_scrape)
    if not df2.empty:
        return df2

    return pd.DataFrame()


# In LinkedIn, one page contains 25 jobs by default. This function tries to navigate to the next page by clicking the "Next" button or a specific page number button.
def move_to_next_page(target_page_num: Optional[int] = None, timeout: int = 12) -> bool:
    try:
        wait = WebDriverWait(driver, timeout)
        for _ in range(2):
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);"); time.sleep(0.3)

        clicked = False
        if target_page_num is not None:
            for xp in [
                f'//button[normalize-space()="{target_page_num}"]',
                f'//button[.//span[normalize-space()="{target_page_num}"]]'
            ]:
                try:
                    btn = wait.until(EC.element_to_be_clickable((By.XPATH, xp)))
                    driver.execute_script('arguments[0].scrollIntoView({block:"center"});', btn); time.sleep(0.2)
                    try:
                        btn.click()
                    except (ElementClickInterceptedException, StaleElementReferenceException):
                        driver.execute_script("arguments[0].click();", btn)
                    clicked = True; break
                except TimeoutException:
                    continue

        if not clicked:
            next_btns = driver.find_elements(By.CSS_SELECTOR, 'button[aria-label*="Next"], button[aria-label*="next"]')
            if not next_btns:
                next_btns = driver.find_elements(By.XPATH, '//button[normalize-space()="Next" or .//span[normalize-space()="Next"]]')
            next_btns = [b for b in next_btns if b.is_displayed() and b.is_enabled()]
            if not next_btns:
                return False
            btn = next_btns[0]
            driver.execute_script('arguments[0].scrollIntoView({block:"center"});', btn); time.sleep(0.2)
            try:
                btn.click()
            except (ElementClickInterceptedException, StaleElementReferenceException):
                driver.execute_script("arguments[0].click();", btn)

        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "ul.jobs-search-results__list, .scaffold-layout__list, ul.jobs-search__results-list")))
        time.sleep(0.4)
        if login_wall_present(driver):
            dismiss_login_wall(driver)
        return True
    except (TimeoutException, StaleElementReferenceException):
        return False


JOBS_PER_PAGE = 25
# Build the LinkedIn job search URL based on the provided parameters, including keywords, location, experience level, and posting time.
def build_job_search_url(
    keywords: str = "Computer Science",
    location: str = "United States",
    experience: ExperienceFilter = ExperienceFilter.ALL,
    posted: JobPostTime = JobPostTime.ANY_TIME
) -> str:
    base = "https://www.linkedin.com/jobs/search"
    params = []
    if keywords:
        params.append(f"keywords={quote_plus(keywords)}")
    if location:
        params.append(f"location={quote_plus(location)}")
    if experience.value:
        params.append(f"f_E={experience.value}")
    if posted.value:
        params.append(f"f_TPR={posted.value}")
    params.append("position=1"); params.append("pageNum=0")
    return f"{base}?{'&'.join(params)}"


# Main function to scrape LinkedIn jobs, it could get the specified number of job postings by navigating through multiple pages if necessary.
def scrape_linkedin_jobs(nb_jobs: int, url: str, jobs_per_page: int = None, max_total: int = 1000) -> pd.DataFrame:
    global driver
    if jobs_per_page is None:
        jobs_per_page = JOBS_PER_PAGE
    driver = ensure_window(driver)
    driver = safe_get(driver, url)

    _maybe_accept_cookies_and_check_login(driver)
    if login_wall_present(driver):
        dismissed = dismiss_login_wall(driver)
        if not dismissed:
            if EMAIL and PASSWORD:
                try_login(driver, EMAIL, PASSWORD)
                driver = safe_get(driver, url)
                _maybe_accept_cookies_and_check_login(driver)
                if login_wall_present(driver):
                    dismiss_login_wall(driver)
            else:
                raise TimeoutException("Blocked by login wall and no credentials provided.")

    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, "body")))
    try:
        wait_for_job_items(driver, timeout=20)
    except TimeoutException:
        pass

    total_pages_to_scrape = ceil(nb_jobs / jobs_per_page)
    current_page = 1
    total_df = pd.DataFrame()

    while True:
        remaining = nb_jobs - len(total_df)
        if remaining <= 0 or len(total_df) >= max_total:
            break
        nb_this_page = min(remaining, jobs_per_page)
        print(f"Scraping page {current_page} (need {nb_this_page} jobs from this page)")
        try:
            page_df = scrape_jobs_from_page(nb_this_page)
        except TimeoutException:
            try:
                driver.refresh()
                WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "body")))
                page_df = scrape_jobs_from_page(nb_this_page)
            except Exception:
                page_df = pd.DataFrame()
        if not page_df.empty:
            total_df = pd.concat([total_df, page_df], ignore_index=True)
            if "Link" in total_df.columns:
                total_df.drop_duplicates(subset=["Link"], keep="first", inplace=True, ignore_index=True)
        if current_page >= total_pages_to_scrape or len(total_df) >= nb_jobs or len(total_df) >= max_total:
            break
        moved = move_to_next_page(current_page + 1)
        if not moved:
            print("No further pages found; stopping.")
            break
        current_page += 1
        time.sleep(0.9)

    total_df.reset_index(drop=True, inplace=True)
    return total_df


In [ ]:
_HEADING_MAP = {
    "responsibilities": "resp",
    "what you will do": "resp",
    "what you'll do": "resp",
    "what you do": "resp",
    "duties": "resp",
    "key duties": "resp",
    "role & responsibilities": "resp",
    "role and responsibilities": "resp",
    "requirements": "genreq",
    "must have": "genreq",
    "you have": "genreq",
    "required qualifications": "req",
    "minimum qualifications": "req",
    "basic qualifications": "req",
    "qualifications": "req",
    "preferred qualifications": "pref",
    "preferred": "pref",
    "nice to have": "pref",
    "bonus": "pref",
    "plus": "pref",
}
_heading_regex = re.compile(
    r"^\s*(?:" +
    r"|".join(re.escape(k) for k in sorted(_HEADING_MAP, key=len, reverse=True)) +
    r")\s*:?.*$",
    re.IGNORECASE
)

def _normalize_bullets(text: str) -> str:
    t = re.sub(r"\r\n?", "\n", text or "")
    t = re.sub(r"[•·▪●◦–—\-]\s*", "\n- ", t)
    t = re.sub(r"\n{2,}", "\n", t)
    return t.strip()

def parse_description_sections(text: str):
    text = _normalize_bullets(text)
    lines = [ln.strip() for ln in text.split("\n") if ln.strip()]
    buckets = {"resp": [], "req": [], "pref": [], "genreq": []}
    current = None
    for ln in lines:
        lnl = ln.lower()
        if _heading_regex.match(lnl):
            for key, bucket in _HEADING_MAP.items():
                if key in lnl:
                    current = bucket; break
            continue
        item = ln.lstrip("- ").strip()
        if not item: continue
        if current == "resp" and len(buckets["resp"]) < 60: buckets["resp"].append(item)
        elif current == "req" and len(buckets["req"]) < 60: buckets["req"].append(item)
        elif current == "pref" and len(buckets["pref"]) < 60: buckets["pref"].append(item)
        elif current == "genreq" and len(buckets["genreq"]) < 60: buckets["genreq"].append(item)
    if not any(buckets.values()):
        bullets = [ln.lstrip("- ").strip() for ln in lines if ln.startswith("- ")]
        buckets["req"] = bullets[:30]
    return buckets

# Apply the parsing function to each job description in the DataFrame and create new columns for each section.
def split_description_sections(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    out["Responsibilities"] = [[] for _ in range(len(out))]
    out["QualificationsRequired"] = [[] for _ in range(len(out))]
    out["QualificationsPreferred"] = [[] for _ in range(len(out))]
    out["Requirements"] = [[] for _ in range(len(out))]
    for i, desc in enumerate(out["Description"].fillna("")):
        buckets = parse_description_sections(desc)
        out.at[i, "Responsibilities"] = buckets["resp"]
        out.at[i, "QualificationsRequired"] = buckets["req"]
        out.at[i, "QualificationsPreferred"] = buckets["pref"]
        out.at[i, "Requirements"] = buckets["genreq"]
    return out

# A convenience function that combines building the search URL, scraping the jobs, and parsing the descriptions into a single DataFrame.
def scrape_and_parse_linkedin_jobs(nb_jobs: int,
                                   keywords: str = "Computer Science",
                                   location: str = "United States",
                                   experience: ExperienceFilter = ExperienceFilter.ALL,
                                   posted: JobPostTime = JobPostTime.ANY_TIME) -> pd.DataFrame:
    """Returns a SINGLE DataFrame with Title, Company, Location, Link, Description and parsed columns."""
    url = build_job_search_url(keywords=keywords, location=location, experience=experience, posted=posted)
    df = scrape_linkedin_jobs(nb_jobs=nb_jobs, url=url)
    if df.empty:
        return df
    df = split_description_sections(df)
    # reorder columns for convenience
    cols = ["Title","Company","Location","Link","Description",
            "Responsibilities","QualificationsRequired","QualificationsPreferred","Requirements"]
    return df[[c for c in cols if c in df.columns] + [c for c in df.columns if c not in cols]]


In [11]:
driver = ensure_window(driver); driver = safe_get(driver, "https://www.linkedin.com/jobs/")

final_df = scrape_and_parse_linkedin_jobs(
    nb_jobs=10,                             # Adjust the number of jobs to scrape
    keywords="Computer Science",            # Adjust the keywords to search for
    location="United States",               # Adjust the location to search in
    experience=ExperienceFilter.ALL,
    posted=JobPostTime.ANY_TIME             # Adjust the posting time filter
)

Scraping page 1 (need 10 jobs from this page)


In [ ]:
import re
from pandas.tseries.offsets import DateOffset

# Get correct job locations from mixed information
parts = final_df["Location"].fillna("").str.replace("\n", " · ", regex=False).str.split("·")
final_df["Location"] = parts.str[0].str.strip()
final_df["Posted_text"] = parts.apply(
    lambda xs: next((t.strip() for t in xs if isinstance(xs, list) and ("ago" in t.lower() or "just now" in t.lower())), "")
)

# The jobs posted time in LinkedIn usually looks like "xx days/months/years ago", so we need to use the today's time to get specific date
def to_today_minus(s: str) -> str:
    s = (s or "").lower().strip()
    if "just now" in s:
        return "today"
    m = re.search(r"(\d+)", s)
    if m and "day" in s:
        n = int(m.group(1))
        return "today - " + str(n) + ("day" if n == 1 else "days")
    return ""

final_df["Posted_relative"] = final_df["Posted_text"].apply(to_today_minus)


# This function allow us to use Chicago time to show the posted date in form of "mm/dd/yyyy"
def to_posted_date(s: str, now=None, tz: str = "America/Chicago"):
    if now is None:
        now = pd.Timestamp.now(tz)  # current time with timezone

    s = (str(s) or "").lower().strip()
    if not s:
        return pd.NaT
    if "just now" in s:
        return now.normalize().strftime("%Y/%m/%d")

    m = re.search(r"(\d+)\s*(minute|hour|day|week|month|year)s?\s*ago", s)
    if not m:
        return pd.NaT

    n = int(m.group(1))
    unit = m.group(2)

    if unit == "minute":
        dt = now - pd.Timedelta(minutes=n)
    elif unit == "hour":
        dt = now - pd.Timedelta(hours=n)
    elif unit == "day":
        dt = now - pd.Timedelta(days=n)
    elif unit == "week":
        dt = now - pd.Timedelta(weeks=n)
    elif unit == "month":
        dt = now - DateOffset(months=n)   
    elif unit == "year":
        dt = now - DateOffset(years=n)    
    else:
        return pd.NaT
    return dt.normalize().strftime("%Y/%m/%d")

In [14]:
final_df["Posted_date"] = final_df["Posted_text"].apply(to_posted_date)
final_df.drop(columns=["Posted_text", "Posted_relative"], inplace=True)

In [15]:
final_df

,Title,Company,Location,Link,Description,Responsibilities,QualificationsRequired,QualificationsPreferred,Requirements,Posted_date
0,Data Analyst -- Entry Level,CGI,"Dallas, TX",https://www.linkedin.com/jobs/view/4290918685/...,About the job\nPosition Description\n\nLaunch ...,[],"[What You’ll Bring, Bachelor’s degree from an ...",[],[],2025/08/26
1,Tech Intern,Hewlett Packard Enterprise,"Spring, TX",https://www.linkedin.com/jobs/view/4289266480/...,About the job\nThis role has been designed as ...,"[Management Level Definition:, Support senior ...",[],[],[],2025/08/26
2,Information Security Intern,SoTalent,"Dallas, TX",https://www.linkedin.com/jobs/view/4290763403/...,About the job\nJob Title : Securities Research...,[],[],[Coursework or exposure to cybersecurity / dat...,[],2025/08/28
3,Data Science Intern,Hireshire,,https://www.linkedin.com/jobs/view/4291119840/...,About the job\nAbout HireShire\n\nHireShire is...,[],[],[Knowledge of BI tools (Power BI / Tableau / L...,[Pursuing (or recently completed) B.Tech/BE/M....,NaT
4,Data Science Intern - Summer 2026,Altruist,"Los Angeles, CA",https://www.linkedin.com/jobs/view/4279910921/...,About the job\nAbout Altruist\n\nAltruist is t...,[],[clearing brokerage firm with intuitive softwa...,[],[],2025/08/26
5,Information Security Analyst Intern,"TrueNorth Companies, L.C.","Cedar Rapids, IA",https://www.linkedin.com/jobs/view/4280490167/...,About the job\nAre you looking for an opportun...,[],"[week paid program, running May, August 2026. ...",[],[],2025/08/27
6,Data Analyst,Paramount Pictures,"Los Angeles, CA",https://www.linkedin.com/jobs/view/4279650100/...,About the job\n#WeAreParamount on a mission to...,"[Lead all aspects of the collection, cleaning,...",[Proficiency in data analysis tools and softwa...,[],[],2025/08/26


In [16]:
final_df.to_csv("linkedin_jobs_demo.csv", index=False)